In [1]:
ARITH_BINARY = {'add':'A=A-1, M=M+D,','sub':'A=A-1, M=M-D,','and':'A=A-1, M=M&D,','or':'A=A-1, M=M|D,'}
ARITH_UNARY = {'neg':'@SP, A=M-1, M=-M,','not':'@SP, A=M-1, M=!M,'}
ARITH_TEST  = {'gt':'JGT','lt':'JLT','eq':'JEQ'}
SEGLABEL = {'argument':'ARG','local':'LCL','this':'THIS','that':'THAT'}

LABEL_NUMBER = 0

def pointerSeg(push,seg,index):
    # The following puts the segment's pointer + index in the D
    # register
    instr = '@%s, D=M, @%d, D=D+A,'%(SEGLABEL[seg],int(index))
    if push == 'push':
        return instr + ('A=D, D=M,'+getPushD())
    elif push == 'pop':
        return (instr+'@R15, M=D,' + getPopD() + '@R15, A=M, M=D,')
    else:
        print("Yikes, pointer segment, bet seg not found.")

def fixedSeg(push,seg,index):
    if push == 'push':
        if seg == 'pointer':
            return ('@%d, D=M,'%(3 + int(index)) + getPushD())
        elif seg == 'temp':
            return ('@%d, D=M,'%(5 + int(index)) + getPushD())
    elif push == 'pop':
        if seg == 'pointer':
            return (getPopD() + '@%d, M=D,'%(3 + int(index)))
        elif seg == 'temp':
            return (getPopD() + '@%d, M=D,'%(5 + int(index)))
    else:
        print("Yikes, fixed segment, but seg not found.")

def constantSeg(push,seg,index):
    global filename
    fn_ = filename.split('/')
    fn_ = fn_[-1]
    fn_ = fn_.split('.')
    fn_ = fn_[0]
    if seg == 'constant':
        return '@%d,    D=A,'%(int(index)) + getPushD()
    elif seg == 'static' and push == 'push':
        return '@%s.%d, D=M,'%(fn_,int(index)) + getPushD()
    elif seg == 'static' and push == 'pop':
        return getPopD() + '@%s.%d, M=D,'%(fn_,int(index))

# Yeah this is akward, but needed so that the function handles are known.
SEGMENTS = {'constant':constantSeg,'static':constantSeg,'pointer':fixedSeg,'temp':fixedSeg,\
            'local':pointerSeg,'argument':pointerSeg,'this':pointerSeg,'that':pointerSeg}


def getIf_goto(label):
    """
    Returns Hack ML to goto the label specified as
    an input argument if the top entry of the stack is
    true.
    """
    return getPopD() + '@%s, D;JNE,' % label

def getGoto(label):
    """
    Return Hack ML string that will unconditionally 
    jump to the input label.
    """
    return '@%s, 0;JMP,' % label

def getLabel(label):
    """
    Returns Hack ML for a label, e.g., (label)
    """
    return '(%s),' % label



def _getPushMem(src):
    """
    Helper function to push memory to location src to stack
    """
    ans = "@" + str(src) + ",D=M," + getPushD()
    return ans

def _getPushLabel(src):
    """
    Helper function to push the ROM address of a label to the
    stack.
    """
    ans = "@" + str(src) + ",D=A," + getPushD()
    return ans
   

def _getPopMem(dest):
    """
    Helper function to pop the stack to the memory address dest.
    """
    ans = getPopD() + "@" + str(dest) + ",M=D,"
    return ans

def _getMoveMem(src,dest):
    """
    Helper function to move the contents of src to memory location dest.
    """
    ans = "@" + str(src) + ",D=M," + "@" + str(dest) + ",M=D,"
    return ans

def line2Command(l):
            return l[:l.find('//')].strip()

def uniqueLabel():
    global LABEL_NUMBER
    LABEL_NUMBER +=1
    return "UL_"+ str(LABEL_NUMBER)

def getPushD():
     # This method takes no arguments and returns a string with assembly language
     # that will push the contents of the D register to the stack.
     return '@SP, A=M, M=D, @SP, M=M+1,'

def getPopD():
    # This method takes no arguments and returns a string with assembly language
    # that will pop the stack to the D register.
    # SIDE EFFECT: The A register contains the SP.
     return '@SP, AM=M-1, D=M,'

In [2]:
def theiruniqueLabel(id):
    """ Uses LABEL_NUMBER to generate and return a unique label"""
    global LABEL_NUMBER
    label = f"{id.upper()}{LABEL_NUMBER}"
    LABEL_NUMBER += 1
    return label



In [32]:
def mygetReturn():
    """
    Returns Hack ML code to perform a return, one
    of the more complex operations in this unit.
    The code restores all the memory segments to the
    calling function. It also has to restore the 
    instruction pointer (IP) and reset the stack 
    pointer. See slides 64-76 of nand2tetris.org
    project 8.
    """
    #overall idea
    # // The code below creates and uses two temporary variables:
    # // endFrame and retAddr;
    # // The pointer notation *addr is used to denote: RAM[addr].
    # endFrame = LCL            // gets the address at the frame’s end
    # retAddr = *(endFrame – 5) // gets the return address
    # *ARG = pop()              // puts the return value for the caller
    # SP = ARG + 1              // repositions SP
    # THAT = *(endFrame – 1)    // restores THAT
    # THIS = *(endFrame – 2)    // restores THIS
    # ARG = *(endFrame – 3)     // restores ARG
    # LCL = *(endFrame – 4)     // restores LCL\
    # goto retAddr              // jumps to the return address

    #save endframe = LCL
    code = "@LCL, D=M,@R14, M=D," #save to R14
    #get return address
    code += "@5, A=D-A, D=M, @R15, M=D," #save to R15

    #pop return value
    code += _getPopMem("ARG")

    #reposition SP
    code += "@ARG, D=M+1, @SP, M=D" #SP = ARG + 1

    #wanna use _getPopMem()
    #restore THAT
    code += "@R14, AM=M-1, D=M, @THAT, M=D," #THAT = *(endFrame - 1)
    code += "@R14, AM=M-1, D=M, @THIS, M=D," #THIS = *(endFrame - 2)
    code += "@R14, AM=M-1, D=M, @ARG, M=D," #ARG = *(endFrame - 3)
    code += "@R14, AM=M-1, D=M, @LCL, M=D," #LCL = *(endFrame - 4)

    #goto retAddr
    code += "@R15, A=M, 0;JMP," #goto retAddr

    #have a return label so maybe use that???
    return code

In [33]:
def theirgetReturn():
    """
    Returns Hack ML code to perform a return, one
    of the more complex operations in this unit.
    The code restores all the memory segments to the
    calling function. It also has to restore the
    instruction pointer (IP) and reset the stack
    pointer. See slides 64-76 of nand2tetris.org
    project 8.
    """
    # output_string = "// return,"
    output_string = ""

    # output_string += "// FRAME = LCL,"
    output_string += _getMoveMem("LCL","R13")

    # output_string += "// RET = *(FRAME - 5),"
    output_string += ",".join([
        "@5",
        "A=D-A",
        "D=M",
        "@R14",   #// R!$ =RET",
        "M=D",
        ","
    ])

    # output_string += "// *ARG = pop(),"
    output_string += _getPopMem("ARG")

    # output_string += "// SP = ARG + 1,"
    output_string += ",".join([
        "@ARG",
        "D=M+1",
        "@SP",
        "M=D",
        ","
    ])

    to_restore = ["THAT", "THIS", "ARG", "LCL"]
    for index, item in enumerate(to_restore):
        # output_string += f"// Restore {item} = *(FRAME - {index + 1}),"
        output_string += _getRestore(item)

    # output_string += "// goto RET,"
    output_string += ",".join([
        "@R14",
        "A=M",
        "0;JMP",
        ",",
    ])

    return output_string
def _getRestore(destination):
    return ",".join([
        "@R13",
        "AM=M-1",
        "D=M",
        f"@{destination}",
        "M=D",
        ","
    ])

In [34]:
mygetReturn()

'@LCL, D=M,@R14, M=D,@5, A=D-A, D=M, @R15, M=D,@SP, AM=M-1, D=M,@ARG,M=D,@ARG, D=M+1, @SP, M=D@R14, AM=M-1, D=M, @THAT, M=D,@R14, AM=M-1, D=M, @THIS, M=D,@R14, AM=M-1, D=M, @ARG, M=D,@R14, AM=M-1, D=M, @LCL, M=D,@R15, A=M, 0;JMP,'

In [35]:
theirgetReturn()

'@LCL,D=M,@R13,M=D,@5,A=D-A,D=M,@R14,M=D,,@SP, AM=M-1, D=M,@ARG,M=D,@ARG,D=M+1,@SP,M=D,,@R13,AM=M-1,D=M,@THAT,M=D,,@R13,AM=M-1,D=M,@THIS,M=D,,@R13,AM=M-1,D=M,@ARG,M=D,,@R13,AM=M-1,D=M,@LCL,M=D,,@R14,A=M,0;JMP,,'

In [54]:
def mygetCall(function,nargs):
    """
    This function returns the Hack ML code to 
    invoke the `call function nargs` type command in 
    the Hack virtual machine (VM) language.

    In order for this to work, review slides
    46-58 in the Project 8 presentation available 
    on the nand2tetris.org website.
    """
    #handling call
    #we have to:
    #   save caller's segment pointers
    #   reposition ARG
    #   reposition LCL
    #   Go to execute the callee's code

    #no helper function to set working stack of caller and nArgs

    #---------bot stack----------
    #working stack of caller
    #nArgs
    #saved frame
    #nVars

    #implementation
    #push the return address
    returnLabel = uniqueLabel()
    code = _getPushLabel(returnLabel)

    #push the LCL pointer
    code += _getPushMem('LCL')
    #push the ARG pointer
    code += _getPushMem('ARG')
    #push the THIS pointer
    code += _getPushMem('THIS')
    #push the THAT pointer
    code += _getPushMem('THAT')

    #reposition ARG
    code += "@SP, D=M, @" + str(int(nargs) + 5) + ", D=D-A, @ARG, M=D,"
    #reposition LCL
    code += _getMoveMem("SP", "LCL")
    
    #jump to function
    code += "@" + function + ", 0;JMP,"

    code += getLabel(returnLabel)
    return code

In [58]:
def theirgetCall(function,nargs):
    """
    This function returns the Hack ML code to
    invoke the `call function nargs` type command in
    the Hack virtual machine (VM) language.
    In order for this to work, review slides
    46-58 in the Project 8 presentation available
    on the nand2tetris.org website.
    """
    # outString = f"// call {function} {nargs},"
    outString = ""
    l = theiruniqueLabel("Call_Label")
    segs = ["LCL", "ARG", "THIS", "THAT"]

    outString += _getPushLabel(l)

    for item in segs:
        outString += _getPushMem(item)

    outString += ",".join([
        # "// ARG = SP - nArgs - 5",
        "@SP",
        "D=M",
        f"@{int(nargs) + 5}",
        "D=D-A",
        "@ARG",
        "M=D",
        ","
    ])

    outString += ",".join([
        # "// LCL = SP",
        _getMoveMem("SP", "LCL"),
        # ",// goto Fn",
        f"@{function}",
        "0;JMP",
        f"({l})" #  // (return address)",
        ","
    ])



    return outString

In [59]:
mygetCall("function", 4)

'@UL_12,D=A,@SP, A=M, M=D, @SP, M=M+1,@LCL,D=M,@SP, A=M, M=D, @SP, M=M+1,@ARG,D=M,@SP, A=M, M=D, @SP, M=M+1,@THIS,D=M,@SP, A=M, M=D, @SP, M=M+1,@THAT,D=M,@SP, A=M, M=D, @SP, M=M+1,@SP, D=M, @9, D=D-A, @ARG, M=D,@SP,D=M,@LCL,M=D,@function, 0;JMP,(UL_12),'

In [60]:
theirgetCall("function", 4)

'@CALL_LABEL12,D=A,@SP, A=M, M=D, @SP, M=M+1,@LCL,D=M,@SP, A=M, M=D, @SP, M=M+1,@ARG,D=M,@SP, A=M, M=D, @SP, M=M+1,@THIS,D=M,@SP, A=M, M=D, @SP, M=M+1,@THAT,D=M,@SP, A=M, M=D, @SP, M=M+1,@SP,D=M,@9,D=D-A,@ARG,M=D,,@SP,D=M,@LCL,M=D,,@function,0;JMP,(CALL_LABEL12),'